# Tutorial: X-ray Image Data

This notebook introduces one of the real data sets we'll use in this class, namely an image produced from X-ray CCD data. There's a fair bit of domain-specific information here, but it's useful stuff to see if you haven't worked with imaging data before (regardless of wavelength). Do note that there is also quite a bit of corner cutting, however; the problems based on these data are meant to get key statistical concepts across, and not to show you how to do a completely rigorous analysis that accounts for all instrumental and systematic effects.

In [ ]:
exec(open('tbc.py').read()) # define TBC and TBC_above
import astropy.io.fits as pyfits
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.visualization import LogStretch
logstretch = LogStretch()
import scipy.stats as st

Modern X-ray CCDs are technologically similar to the CCDs used in optical astronomy: when a photon hits a pixel, one or more electrons are promoted into the conduction band and trapped there until being read out. The main practical difference is that X-ray photons are rarer and their energies much higher.

This means that:
* Only for exceptionally bright sources will we ever have $>1$ photon hit a given pixel in an integration, if we read out the CCD every few seconds.
* We do not get 1 electron promoted per photon, as is the case in visible wavelength CCDs. Instead, the number of electrons is roughly proportional to the photon's energy, which means that these imaging devices are actually imaging spectrometers.
* When we say "counts" in this context, we mean "pixel activation events" rather than number of electrons trapped, so that (as in optical astronomy) we're referring to the number of photons detected (or other events that look like photons to the detector).

Let's look at some processed data from XMM-Newton for galaxy cluster Abell 1835.

Here the raw "event list" of pixel activations has been processed to form an image, meaning that, other than a broad selection on photon energy, _the spectral information has been discarded_.

XMM actually has 3 CCD cameras, but we'll just work with 1 for simplicity, and with just one of the available observations.

We'll still need 2 files:
* The image, in units of counts
* The exposure map (units of seconds), which accounts for the exposure time and the variation in effective collecting area across the field due to vignetting

There's a nice web interface that allows one to search the public archives for data, but to save time, just download the particular image and exposure map here:
* https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0098010101/PPS/P0098010101M2U009IMAGE_3000.FTZ
* https://heasarc.gsfc.nasa.gov/FTP/xmm/data/rev0/0098010101/PPS/P0098010101M2U009EXPMAP3000.FTZ

This is an image produced from 1-3 keV events captured by the MOS2 camera in XMM's first observation of A1835, way back in 2001, and the corresponding exposure map.

You can put these files anywhere you want; I will assume they live in a directory called `ignore`, one level up from `tutorials`.

In [ ]:
TBC() # datadir = '../ignore/' # or whatever - path to where you put the downloaded files, including the trailing '/'

Both files are in FITS image format, which we can read in using `astropy.io.fits` (here aliased to its older name, `pyfits`).

In [ ]:
imagefile = datadir + 'P0098010101M2U009IMAGE_3000.FTZ'
expmapfile = datadir + 'P0098010101M2U009EXPMAP3000.FTZ'

imfits = pyfits.open(imagefile)
exfits = pyfits.open(expmapfile)

Let's see what we've got:

In [ ]:
imfits.info()

In [ ]:
exfits.info()

`imfits` is a FITS object, containing multiple data structures. The image itself is an array of integer type (remember, counts!), and size 648x648 pixels, stored in the primary "header data unit" or HDU, and accessed via the `data` method of the FITS object. The other HDUs hold tables containing the "good time intervals" defined in earlier data processing, which we can ignore for our purposes.

`exfits` contains only the exposure map, with floating point type.

Here we extract the image (that is, the array) data from each object as `numpy` arrays.

In [ ]:
im = imfits[0].data
ex = exfits[0].data
print(im.shape, im.dtype)
print(ex.shape, ex.dtype)

Note: If we wanted `im` to be floating point for some reason, we would need to cast it, as in `im = imfits[0].data.astype('np.float32')`.

Let's have a look the image and exposure map. It's often helpful to stretch images on a logarithmic scale because some sources can differ in brightness by orders of magnitude. The exposure map varies much less, so a linear scale works better in that case.

Some more details: FITS images (and the arrays we read from them) are indexed according to an ancient convention, whereby the first index corresponds to the vertical axis (line) and the second index corresponds to the horizontal axis (sample). This corresponds to the way `matplotlib` interprets arrays as images, although we need to use the `origin='lower'` option to display the image the right way up.

In [ ]:
plt.rcParams['figure.figsize'] = (20.0, 10.0)
fig, ax = plt.subplots(1,2);
ax[0].imshow(logstretch(im), cmap='gray', origin='lower');
ax[0].set_title('image (log scale)');
ax[1].imshow(ex, cmap='gray', origin='lower');
ax[1].set_title('exposure map');

Note that information from 7 different CCDs in the MOS2 camera have been combined here, and that X and Y in the image arrays correspond to celestial coordinates (right ascension and declination) rather than X and Y on a given detector or in the focal plane.

In the image, we can see:
1. Galaxy cluster Abell 1835 (the big blob in the center).
2. Various other sources (smaller blobs). These are point-like sources - mostly active galactic nuclei (AGN) - that have been smeared out by the telescope's point spread function (PSF).
3. A roughly uniform background, consisting of unresolved AGN, diffuse X-rays from the Galactic halo and local hot bubble,  and events due to particles (solar wind protons and cosmic rays) interacting with the CCD.

The exposure map shows:
1. Clear boundaries between the 7 CCDs that make up the MOS2 camera, and a number of "bad rows/columns" where the exposure has been set to zero.
2. An overall gradient with radius - this is the vignetting function of the telescope.
3. A vaguely circular cut-out shape along the edge. This is applied in preprocessing to eliminate pixels where the effective exposure is essentially zero. All of the CCDs are, in fact, square, and the "corner" regions of the field of view are sometimes used to get a measurement of the portion of the background that is not focussed by the optics (e.g. particle-induced events).

You probably know that, in python, array indices start from 0. We could choose to work with $x$ and $y$ coordinates that are simply these indices. However, conventionally, astronomical image coordinates are indexed from 1; for example, if we had used the tool `ds9` to define a region of interest in this image, and saved it in "Image" coordinates (as opposed to celestial coordinates), the bottom-left pixel in the image would be $(1,1)$. To avoid confusion, we might want to follow this convention.

Below is a simple class that should assist in displaying these data, and potentially cutting out "sub-images" for local analysis. It holds on to the image and exposure map, and also defines arrays `imx` and `imy` which hold the Image X and Y coordinates of each pixel. (Note: we will use "pixel" to refer to an entry in these arrays, as opposed to a physical pixel in one of the CCDs.)

In [ ]:
class Image:
    def __init__(self, imdata, exdata, imx=None, imy=None):
        self.im = imdata
        self.ex = exdata
        if imx is None or imy is None:
            # add 1 to make IMAGE coordinates
            self.imx, self.imy = np.meshgrid(np.arange(imdata.shape[0])+1, np.arange(imdata.shape[1])+1)
        else:
            self.imx = imx
            self.imy = imy
    def cutout(self, x0, x1, y0, y1):
        # Again note that the arguments are meant to be IMAGE coordinates, indexed starting from 1
        return Image(self.im[(y0-1):y1,(x0-1):x1], self.ex[(y0-1):y1,(x0-1):x1], 
                     self.imx[(y0-1):y1,(x0-1):x1], self.imy[(y0-1):y1,(x0-1):x1])
    def extent(self):
        return [np.min(self.imx), np.max(self.imx), np.min(self.imy), np.max(self.imy)]
    def display(self, log_image=True):
        fig, ax = plt.subplots(1,2);
        extent = self.extent()
        if log_image:
            ax[0].imshow(logstretch(self.im), cmap='gray', origin='lower', extent=extent);
            ax[0].set_title('image (log scale)');
        else:
            ax[0].imshow(self.im, cmap='gray', origin='lower', extent=extent);
            ax[0].set_title('image');
        ax[1].imshow(self.ex, cmap='gray', origin='lower', extent=extent);
        ax[1].set_title('exposure map');

Let's create such an object holding the original data:

In [ ]:
orig = Image(im, ex)

To quickly illustrate what `imx` and `imy` are for, let's look at them:

In [ ]:
plt.rcParams['figure.figsize'] = (10.0, 10.0)
fig, ax = plt.subplots(1,2);
ax[0].imshow(orig.imx, cmap='gray', origin='lower');
ax[0].set_title('imx');
ax[1].imshow(orig.imy, cmap='gray', origin='lower');
ax[1].set_title('imy');

Keeping these arrays around is obviously a little inefficient in terms of memory, but it means we can easily do calculations involving pixel positions with `numpy` array arithmetic.

Next, let's make a cut-out (or "postage stamp") around roungly the center of the image, and look at it.

In [ ]:
stamp = orig.cutout(300, 400, 300, 400)

plt.rcParams['figure.figsize'] = (10.0, 10.0)
stamp.display()

Note that `imx` and `imy` in `stamp` hold the Image coordinates of each pixel with respect to the _original_ image, which seems like a potentially useful thing.

Something we might want to do when evaluating a model is to compute the distance of each pixel from some specified position, in units of pixels. As a quick test, complete the following to compute an array holding the distance of each pixel in the `stamp` image from the center of original image. (Remember that Image coordinates start from 1, not 0!)

`dist=0` should fall within the stamp, in the bottom-left quadrant as displayed below.

In [ ]:
TBC() # dist = something involving stamp.imx and stamp.imy

In [ ]:
plt.rcParams['figure.figsize'] = (5.0, 5.0)
plt.imshow(dist, cmap='gray', origin='lower', extent=stamp.extent());

So now we can easily compute the distance between all pixels and some reference, in units of the pixel size. Note that the Euclidean distance formula is not exact, since the sky is a sphere and not a plane, but on this scale it's easily good enough. Which prompts the question - just what is the size of a pixel in this image?

To find out, we can consult the FITS header of the image. The relevant keywords are near the bottom of the header, and begin `CTYPE`, `CRPIX`, `CVAL`, etc. Note that the exact keywords can vary, since FITS files can have multiple coordinate systems defined. In this case, the header tells us that axis 1 (X) is Right Ascension, axis 2 (Y) is DEClination, and both have a pixel length of 0.0011111... in units of degrees, so 4 arcseconds. (CDELT1 is negative because RA increases to the left by convention.)

In [ ]:
imfits[0].header

If we need to, header values can be extracted easily. This extracts the pixel size in the X direction, and converts it to arcseconds (`CUNIT1` tells us that it is originally in units of degrees).

In [ ]:
imfits[0].header['CDELT1'] * 3600

We'll pick this up in the next tutorial notebook, [`agn_photometry_grid`](agn_photometry_grid.ipynb), and then continue in the other `agn_photometry_*` tutorials.